In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# train = pd.read_csv("/content/drive/MyDrive/train_new_upto_5K.csv")
train = pd.read_csv("/content/drive/MyDrive/train_full.csv")
# val = pd.read_csv("/content/drive/MyDrive/val_full.csv")
# test = pd.read_csv("/content/drive/MyDrive/test_full.csv")

In [ ]:
train = train[:5000]

In [ ]:
train

,Unnamed: 0,text_status,text,languages,image_url_status,image_url,video_thumbnail_status,video_thumbnail_url,video_url_status,video_url,...,year,month,day_of_year,day_of_week,date,username,inferred company,likes,text_topics,Tag
0,0,Has text,"Spend your weekend morning with a Ham, Egg, an...",['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Eo8N3JLVoAAlDJT?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2020.0,December,347.0,Saturday,2020-12-12 00:47:00,TimHortonsPH,tim hortons,1,food_&_dining,['food_&_dining']
1,1,Has text,Watch rapper <mention> freestyle for over an H...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Dg7mW-VX0AE_hMn?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2018.0,June,181.0,Saturday,2018-06-30 10:04:20,IndyMusic,independent,2750,['music'],['news_&_social_concern']
2,2,Has text,Canadian Armenian community demands ban on mil...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/EjG2s4aXgAMNM1o?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2020.0,September,273.0,Tuesday,2020-09-29 19:47:28,CBCCanada,cbc,57,"['business_&_entrepreneurs', 'news_&_social_co...",['news_&_social_concern']
3,3,Has text,"1st in Europe to be devastated by COVID-19, It...",['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/EjPaVniX0AAaWLJ?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2020.0,October,275.0,Thursday,2020-10-01 11:40:09,MKWilliamsRome,williams,152,['news_&_social_concern'],['sports']
4,4,Has text,An 85-year-old primary school in Shanghai has ...,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/amplify_video_thumb/1322...,Link exists and is accessible,https://video.twimg.com/amplify_video/13223822...,...,2020.0,November,320.0,Sunday,2020-11-15 16:01:08,cnni,cnn,525,"['learning_&_educational', 'youth_&_student_li...",['news_&_social_concern']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4995,Has text,Let AAA Travel’s team make your dream vacation...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Emo5iIvWEAIhQ6a?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2020.0,November,317.0,Thursday,2020-11-12 17:14:02,AAASouthPenn,aaa,0,['travel_&_adventure'],['travel_&_adventure']
4996,4996,Has text,Welcome to the #Get2GetherExperience 🔥💃🏾💃🏾🕺🏾🕺🏾...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/EgiXVd0WoAgMRbL?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2020.0,August,241.0,Friday,2020-08-28 21:12:45,METROFMSA,sabc,213,['music'],['music']
4997,4997,Has text,Quentin Tarantino confirms full cast list for ...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Dlx2MM4W0AAm0_G?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2018.0,August,241.0,Wednesday,2018-08-29 15:55:43,TheIndyFilm,independent,191,"['celebrity_&_pop_culture', 'film_tv_&_video']",['news_&_social_concern']
4998,4998,Has text,Success Tips: 10 Essential Truths to Achieving...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/D9mAN9bX4AAY5SK?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,...,2019.0,June,172.0,Friday,2019-06-21 15:00:48,ToyotaSD,toyota,3,['business_&_entrepreneurs'],['sports']


In [ ]:
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import expit


MODEL = f"cardiffnlp/tweet-topic-21-multi"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

# PT
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
class_mapping = model.config.id2label

tokenizer_config.json:   0%|          | 0.00/1.30k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.88k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [ ]:
def text_topics(text):
  tokens = tokenizer(text, return_tensors='pt')
  output = model(**tokens)

  scores = output[0][0].detach().numpy()
  scores = expit(scores)
  predictions = (scores >= 0.5) * 1

  l = []
  for i in range(len(predictions)):
    if predictions[i]:
      l.append(class_mapping[i])
  return l

In [ ]:
from joblib import parallel_backend

with parallel_backend('threading', n_jobs=2):
  for i in range(m):
    train.at[i,'text_topics'] = text_topics(train['text'][i])
    if(i%50==0):
      print(i, " done")

In [ ]:
((train['text_topics'].isna()==False).sum())

5000

In [ ]:
train['AllTags'] = train.groupby('inferred company')['text_topics'].transform(lambda x: ','.join(x.astype(str)))

train['Tag'] = train['AllTags'].apply(lambda x: pd.Series(x.split(',')).mode()[0])

print(train[['inferred company', 'Tag']])

<ipython-input-33-b3d82342be7a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['AllTags'] = train.groupby('inferred company')['text_topics'].transform(lambda x: ','.join(x.astype(str)))


     inferred company                        Tag
0         tim hortons          ['food_&_dining']
1         independent  ['news_&_social_concern']
2                 cbc  ['news_&_social_concern']
3            williams                 ['sports']
4                 cnn  ['news_&_social_concern']
...               ...                        ...
4995              aaa     ['travel_&_adventure']
4996             sabc                  ['music']
4997      independent  ['news_&_social_concern']
4998           toyota                 ['sports']
4999    credit suisse      'travel_&_adventure']

[5000 rows x 2 columns]


<ipython-input-33-b3d82342be7a>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Tag'] = train['AllTags'].apply(lambda x: pd.Series(x.split(',')).mode()[0])


In [ ]:
train.drop(columns = "AllTags", inplace = True)

<ipython-input-35-6398f515cbf3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.drop(columns = "AllTags", inplace = True)


In [ ]:
train.to_csv("train_new_upto_5K.csv")

##Binning

In [ ]:
train_set_1 = []
train_set_2 = []
train_set_3 = []
train_set_4 = []

In [ ]:
train_log_labels = []
for i in range(len(train['likes'])):
  train_log_labels.append(np.log(1+train['likes'][i]))

In [ ]:
for i in range(len(train_log_labels)):
  if train_log_labels[i] <= np.log(100):
    train_set_1.append(i)


  elif train_log_labels[i] > np.log(100) and train_log_labels[i] <= np.log(3500):
    train_set_2.append(i)


  elif train_log_labels[i] > np.log(3500) and train_log_labels[i] <= np.log(10000):
    train_set_3.append(i)

  else:
    train_set_4.append(i)

In [ ]:
subset1 = train.iloc[train_set_1]
subset2 = train.iloc[train_set_2]
subset3 = train.iloc[train_set_3]
subset4 = train.iloc[train_set_4]

In [ ]:
subset4

,text_status,text,languages,image_url_status,image_url,video_thumbnail_status,video_thumbnail_url,video_url_status,video_url,video_bitrate,...,gif_url,gif_bitrate,year,month,day_of_year,day_of_week,date,username,inferred company,likes
345,Has text,THEY WANT TO BAN FRIED CHICKEN \n\nThese Loone...,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/ext_tw_video_thumb/11870...,Link exists and is accessible,https://video.twimg.com/ext_tw_video/118708494...,256000,...,GIF does not exist,Does not exist,2019.0,October,296.0,Wednesday,2019-10-23 19:15:38,w_terrence,williams,21722
401,Has text,JUST IN CASE YOU MISSED IT \n\nI called out Le...,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/ext_tw_video_thumb/11875...,Link exists and is accessible,https://video.twimg.com/ext_tw_video/118750001...,432000,...,GIF does not exist,Does not exist,2019.0,October,298.0,Friday,2019-10-25 16:47:40,w_terrence,williams,17613
473,Has text,LEGEND\n\n#7heChamp10ns🏆 <hyperlink>,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/media/Db-sgUjU8AA8W6k.jpg,Link exists and is accessible,https://video.twimg.com/ext_tw_video/990700120...,256000,...,GIF does not exist,Does not exist,2018.0,April,119.0,Sunday,2018-04-29 21:11:38,FCBarcelona,fc barcelona,24157
572,Has text,That feeling when you don't have to play again...,['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/D_sJc2sXsAA2KW-?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,Does not exist,...,GIF does not exist,Does not exist,2019.0,July,198.0,Wednesday,2019-07-17 16:10:15,FCBarcelona,fc barcelona,18569
726,Has text,Even the cutest of emojis have no problem ghos...,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/media/D_s7wBPUwAEGBjm.jpg,Link exists and is accessible,https://video.twimg.com/amplify_video/11515791...,432000,...,GIF does not exist,Does not exist,2019.0,July,198.0,Wednesday,2019-07-17 19:50:08,SpiderManMovie,no,34515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
208310,Has text,".<mention>, who recently appeared in an episod...",['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Eo-VUo_W4Ace4UG?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,Does not exist,...,GIF does not exist,Does not exist,2020.0,December,346.0,Friday,2020-12-11 17:39:02,gucci,gucci,31210
208382,Has text,"As MJ would say, it's kind of obvious... we’re...",['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/Eg16JTTVoAAV7WH?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,Does not exist,...,GIF does not exist,Does not exist,2020.0,September,245.0,Tuesday,2020-09-01 16:17:44,SpiderManMovie,no,20688
208443,Has text,We’re giving away 60¢ short stacks on July 17 ...,['ENGLISH'],Link does not exist,Image does not exist,Link exists and is accessible,https://pbs.twimg.com/media/DhrXeS7U0AEiUxa.jpg,Link exists and is accessible,https://video.twimg.com/amplify_video/10163610...,288000,...,GIF does not exist,Does not exist,2018.0,July,190.0,Monday,2018-07-09 16:59:00,IHOP,ihop,62177
208660,Has text,"good morning FT, say it back... <hyperlink>",['ENGLISH'],Link exists and is accessible,https://pbs.twimg.com/media/EfvWbY5X0AEQrY_?fo...,Link does not exist,Video thumbnail does not exist,Link does not exist,Video does not exist,Does not exist,...,GIF does not exist,Does not exist,2020.0,August,232.0,Wednesday,2020-08-19 09:00:00,bayer04_en,bayer,15396


## Latent Dirichlet allocation

In [ ]:
%pip install nltk gensim

In [ ]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from gensim import corpora, models

# Download NLTK resources
nltk.download('stopwords')

In [ ]:
# Sample tweet data
tweets = subset2['text'].values.tolist()

# Preprocess tweets
def preprocess_tweet(tweet):
    # Remove URLs, special characters, and convert to lowercase
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    tweet = tweet.lower()
    return tweet

preprocessed_tweets = [preprocess_tweet(tweet) for tweet in tweets]

# Tokenize tweets using TweetTokenizer
tokenizer = TweetTokenizer()
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in preprocessed_tweets]

# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tweets = [[word for word in tweet if word not in stop_words] for tweet in tokenized_tweets]

# Create a dictionary and a corpus
dictionary = corpora.Dictionary(filtered_tweets)
corpus = [dictionary.doc2bow(tweet) for tweet in filtered_tweets]

# Train LDA model
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# Print topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.089*"hyperlink" + 0.007*"president" + 0.007*"says" + 0.006*"trump" + 0.005*"people"')
(1, '0.100*"mention" + 0.098*"hyperlink" + 0.007*"amp" + 0.005*"day" + 0.005*"time"')
(2, '0.094*"hyperlink" + 0.012*"get" + 0.009*"new" + 0.008*"amp" + 0.005*"know"')


In [ ]:
subset1

In [ ]:
# Sample tweet data
tweets = subset1['text'].values.tolist()

# Preprocess tweets
def preprocess_tweet(tweet):
    # Remove URLs, special characters, and convert to lowercase
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'[^a-zA-Z\s]', '', tweet)
    tweet = tweet.lower()
    return tweet

preprocessed_tweets = [preprocess_tweet(tweet) for tweet in tweets]

# Tokenize tweets using TweetTokenizer
tokenizer = TweetTokenizer()
tokenized_tweets = [tokenizer.tokenize(tweet) for tweet in preprocessed_tweets]

# Remove stopwords
stop_words = set(stopwords.words('english'))
filtered_tweets = [[word for word in tweet if word not in stop_words] for tweet in tokenized_tweets]

# Create a dictionary and a corpus
dictionary = corpora.Dictionary(filtered_tweets)
corpus = [dictionary.doc2bow(tweet) for tweet in filtered_tweets]

# Train LDA model
lda_model = models.LdaModel(corpus, num_topics=2, id2word=dictionary, passes=15)

# Print topics
topics = lda_model.print_topics(num_words=5)
for topic in topics:
    print(topic)

(0, '0.099*"hyperlink" + 0.043*"mention" + 0.008*"get" + 0.007*"new" + 0.007*"amp"')
(1, '0.145*"hyperlink" + 0.006*"covid" + 0.003*"new" + 0.002*"police" + 0.002*"aaa"')
